# Training MLP model on insurance claims data

### Model

In [ ]:
indices = (1,2)
dims = (2,3)

for (idx,i) in enumerate(indices):
    if i not in list(range(dims[idx])):
        print("error")


In [1]:
from dydx.layers import Linear, Embedding
from dydx.model import Model
from dydx.dataset import xcols_types,xcols
from dydx.download import encodings_dims, encodings_dims


SCALER = 2
EMBEDDING_DIM = 32
cat_data_size  = {k:v for (k,v) in encodings_dims.items() if v > 2 }

embedders =[ (f'e{idx}',Embedding(dims=(in_size, EMBEDDING_DIM))) for (idx,in_size) in  enumerate(cat_data_size.values())]


l1 = Linear( random=True, dims=(4*EMBEDDING_DIM + 6, SCALER*64))

l2 = Linear( random=True,dims=(SCALER*64, SCALER*64))
l3 = Linear( random=True, dims=(SCALER*64, SCALER*32))
l4 = Linear( random=True,dims=(SCALER*32, 16))
l5 = Linear( activation=False, random=True, dims=(16,1)
) # when activation is set to false we use sigmoid function to normalised output 

		
names = ["l1","l2","l3","l4", "l5"]		
layers =[l1,l2,l3,l4, l5]
ls = {**dict(embedders), **dict(zip(names,layers))}
model_fc1 = Model(dict(zip(names,layers)))
model = Model(ls)
print("Number of parameters:", len(model.parameters()))

print(encodings_dims)
from pprint import pprint 
pprint(list(zip(xcols,xcols_types)))
print({k:v for (k,v) in encodings_dims.items() if v > 2 })
# print(xcols)
model.__dict__

layer init seed: 6823321540184574520
dataset seed: 3018045280055472080
/home/ola/Code/dydx/dydx/data
Number of parameters: 47777
{'Agency': 16, 'Agency Type': 2, 'Destination': 102, 'Distribution Channel': 2, 'Gender': 3, 'Product Name': 25}
[('Age', 'ordinal'),
 ('Agency', 'categorical'),
 ('Agency Type', 'binary'),
 ('Commision (in value)', 'ordinal'),
 ('Destination', 'categorical'),
 ('Distribution Channel', 'binary'),
 ('Duration', 'ordinal'),
 ('Gender', 'cateogrical'),
 ('Net Sales', 'ordinal'),
 ('Product Name', 'categorical')]
{'Agency': 16, 'Destination': 102, 'Gender': 3, 'Product Name': 25}


{'ln': dict_keys(['e0', 'e1', 'e2', 'e3', 'l1', 'l2', 'l3', 'l4', 'l5']),
 'e0': <dydx.layers.Embedding at 0x790a875a83b0>,
 'e1': <dydx.layers.Embedding at 0x790a701444d0>,
 'e2': <dydx.layers.Embedding at 0x790a661e9220>,
 'e3': <dydx.layers.Embedding at 0x790a661eb6e0>,
 'l1': <dydx.layers.Linear at 0x790a875a89e0>,
 'l2': <dydx.layers.Linear at 0x790a67ef2840>,
 'l3': <dydx.layers.Linear at 0x790a67513140>,
 'l4': <dydx.layers.Linear at 0x790a6713ad80>,
 'l5': <dydx.layers.Linear at 0x790a671d4800>}

### Data 

In [2]:
from dydx.metrics import Loss
from dydx.dataset import DataLoader, Dataset
from pathlib import Path
from functools import reduce
loss = Loss()
EPOCHS= 1000


ds = Dataset(testing=False) # takes 4048 examples to overfit model too for testing purposes
fname = f"{list(set(model.seed()))[0]}_{ds.seed}.data"
fp =  Path().cwd() / "experiments" / fname
splits = ['train','val','test']
print(f"dataset sizes for {', '.join(splits)} are {[ds.__len__(split) for split in splits]} respectively.")
dst = ds.train
dsv = ds.val
dss = ds.test
# iterators 
dltrain  = DataLoader(dst,32)
dlval = DataLoader(dsv,32)
dltest = DataLoader(dss,23)
for (idx,xy) in enumerate( dltrain()):
    x,y = xy
    if idx == 0:
        pass
        # Agency
        # a_in = Array([x.T().values[:][1]]).T()
        # x0 = model.e0(a_in)
        # # Destination
        # d_in = Array([x.T().values[:][4]]).T()
        # x1 = model.e1(d_in)
        # # Gender
        # g_in = Array([x.T().values[:][7]]).T()
        # x2 = model.e2(g_in)
        # # Product Name
        # p_in = Array([x.T().values[:][9]]).T()
        # x3 = model.e3(p_in)
        
        # x4 = Array([x.T().values[:][0]]).T()
        # x5 = Array([x.T().values[:][2]]).T()
        # x6 = Array([x.T().values[:][3]]).T()
        # x7 = Array([x.T().values[:][5]]).T()
        # x8 = Array([x.T().values[:][6]]).T()
        # x9 = Array([x.T().values[:][8]]).T()
        
        
        # features = [x0, x1, x2, x3, x4, x5, x6, x7, x8, x9]
        # stack = lambda x, y : x.stack(y,dim=0)
        # x = reduce(stack, features)
        # x = model.l1(x)
        # x = model.l2(x)
        # x = model.l3(x)
        # x = model.l4(x)
        # y_hat = model.l5(x)
        
        
def propagate(x,model):    
    a_in = x[:,1]
    x0 = model.e0(a_in)
    # Destination
    d_in = x[:,4]
    x1 = model.e1(d_in)
    # Gender
    g_in = x[:,7]
    x2 = model.e2(g_in)
    # Product Name
    p_in = x[:,9]
    x3 = model.e3(p_in)
    x4 = x[:,0]
    x5 = x[:,2]
    x6 = x[:,3]
    x7 = x[:,5]
    x8 = x[:,6]
    x9 = x[:,8]
    features = [x0, x1, x2, x3, x4, x5, x6, x7, x8, x9]
    stack = lambda x, y : x.stack(y,dim=0)
    x = reduce(stack, features)
    x = model.l1(x)
    x = model.l2(x)
    x = model.l3(x)
    x = model.l4(x)
    y_hat = model.l5(x)
    
    return y_hat, model



def forward(self,x):    
    a_in = x[:,1]
    x0 = self.e0(a_in)
    # Destination
    d_in = x[:,4]
    x1 = self.e1(d_in)
    # Gender
    g_in = x[:,7]
    x2 = self.e2(g_in)
    # Product Name
    p_in = x[:,9]
    x3 = self.e3(p_in)
    x4 = x[:,0]
    x5 = x[:,2]
    x6 = x[:,3]
    x7 = x[:,5]
    x8 = x[:,6]
    x9 = x[:,8]
    features = [x0, x1, x2, x3, x4, x5, x6, x7, x8, x9]
    stack = lambda x, y : x.stack(y,dim=0)
    x = reduce(stack, features)
    x = self.l1(x)
    x = self.l2(x)
    x = self.l3(x)
    x = self.l4(x)
    y_hat = self.l5(x)
    
    return y_hat

# model.forward = forward

dataset sizes for train, val, test are [17441, 4983, 2492] respectively.


### Fitting

In [ ]:
EARLY_STOPPING = None
losses = []
OPTMISATION = ['sgd', 'sgd+m'][1]
fname = f"{OPTMISATION}_{list(set(model.seed()))[0]}_{ds.seed}.data"
fp =  Path().cwd() / "experiments" / fname
#print(f'Number of parameterers: {len(model.parameters().parameters)}')

delta_t_minus1 = [0] *  len(model.parameters())
step = 0 
for epoch in range(EPOCHS):
    # print("Training".center(70,"#"))
    for (idx,xy) in enumerate( dltrain()):
        x,y = xy
        
        # y_hat =model(x)		
        y_hat, model = propagate(x,model)
        acc, avg_loss = loss(y,y_hat)

        avg_loss.backward()

        losses.append(avg_loss.data)
        info  = f'Step {step} Loss:{avg_loss.data} train accuracy:{acc*100}%'
        with open(fp, "a+") as f:
            f.write(info + '\n')
  
        print(info)
        
        # optimizer step
        LR = 0.0000001
        if OPTMISATION == 'sgd+m':
            # SGD with momentum
            BETA = 0.9
            for idx, p in enumerate(model.parameters()):
                # if idx in [1,2,3,4]:
                # 	print('p.data:', p.data,  'p.grad:',p.grad)
                p.data = p.data -  LR *( BETA*p.grad + (1-BETA)*delta_t_minus1[idx])
                delta_t_minus1[idx] =  p.grad + (1-BETA)*delta_t_minus1[idx]
        if OPTMISATION == 'sgd':
            # SGD 
            for idx, p in enumerate(model.parameters()):
                # if idx in [1,2,3,4]:
                # 	print('p.data:', p.data,  'p.grad:',p.grad)
                p.data = p.data -  LR *p.grad
        model.zero_grad()
        step += 1 

        # print("After zero-ing grad")
        # for idx,p in enumerate(model.parameters()):
        # 	if idx in [1,2,3,4]:
        # 		print('p.grad:',p.grad)			      
        # # checking if gradients were actually set to zero 
        # for p in model.parameters():
        # 	print('p.grad:',p.grad)

# 	print("Validation".center(70,"#"))
# 	for (idx,xy) in enumerate( dlval()):
# 		x,y = xy 
# #			for layer in model.layers:
# #				x = layer(x)
# 		y_hat = model(x)
# 		acc, avg_loss= loss(y,y_hat)
# 		print(f'Epoch {epoch}:{idx} Loss:{avg_loss.data} val accuracy:{acc*100}%')	

# train()

### XOR gate

In [ ]:
from dydx.metrics import Loss
from dydx.linear_algebra import Array
from dydx.layers import Linear
from dydx.dydx import Scalar
from dydx.model import Model
loss = Loss()

# layer seed: 8433248431303025905
# dataset seed: 3588574048688484214
X = Array([[Scalar(0.),Scalar(0.)],
           [Scalar(0.),Scalar(1.)],
           [Scalar(1.),Scalar(0.)],
           [Scalar(1.),Scalar(1.)]])

y  = Array([[Scalar(0.)],
            [Scalar(1.)], 
            [Scalar(1.)],
            [Scalar(0.)]])

l1 = Linear( random=True,dims=(2, 16))
l2 = Linear( random=True, dims=(16, 16))
l3 = Linear( random=True,dims=(16, 1))
layers = ['l1', 'l2', 'l3']
model = Model(dict(zip(layers,[l1,l2,l3])))

losses= []
EPOCHS = 250_000_0000
for epoch in range(EPOCHS):
	# print("Training".center(70,"#"))
	y_hat = model(X)
	acc, avg_loss = loss(y,y_hat)

	avg_loss.backward()

	losses.append(avg_loss.data)
	
	info  = f'Epoch {epoch} Loss:{avg_loss.data} train accuracy:{acc*100}%'
	print(info)

	lr = 0.0000001
	for idx, p in enumerate(model.parameters()):
		# if idx in [1,2,3,4]:
		# print('p.data:', p.data,  'p.grad:',p.grad)
		p.data = p.data -  lr *p.grad
  
  
	model.zero_grad()

### Torch-equivalent model fitting


In [ ]:
from dydx.dataset import DataLoader, Dataset
ds = Dataset(testing=True) # takes 160 examples to overfit model too for testing purposes
splits = ['train','val','test']
print(f"dataset sizes for {', '.join(splits)} are {[ds.__len__(split) for split in splits]} respectively.")
dst = ds.train
dsv = ds.val
dss = ds.test
# iterators 
dltrain  = DataLoader(dst,32)
dlval = DataLoader(dsv,32)
dltest = DataLoader(dss,23)


import torch.nn as nn
from torch.optim import SGD
import torch.nn.functional as F
from torch import Tensor


# PyTorch models inherit from torch.nn.Module
class ClaimsClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 32)
        self.fc5 = nn.Linear(32,1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.sigmoid(x)
    

model = ClaimsClassifier()

for _ in range(100000):
    optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
    for xy in dltrain():
        x, y = xy
        x = Tensor([[x_.data for x_ in row] for row in x.values])
        y = Tensor([y_.data for y_ in y.values])
        
        #print(Tensor(x))
        #print(Tensor(y))

        optimizer.zero_grad()
        y_hat = model(x)
        y_hat = y_hat.squeeze(1)
        #print(y_hat.shape)
        #print(y_hat)
        loss= F.binary_cross_entropy(y_hat,y)
        loss.backward()
        optimizer.step()
        print(loss)


### Singular value decomposition

In [ ]:
def flatten(v):
    for i in v:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i
            

STEPS = 100
STEP_SIZE = 0.0001


def singular_value_decomposition(m):
    '''
    m = U @ S @ V.T()
    
    decompose matrix m of dimensions (m,n) into:
    U: Unitary matrix of (m,m) i.e. U_inv = U.T() and therefore Det(U) = 1 and therefore cols of U form orthonormal basis of m 
    S: Rectangular diagonal matrix of (m,n)
    V: Unitary matrix of (n,n) i.e. V_inv = V.T() and therefore Det(U) = 1 and therefore cols of V form orthonormal basis of n
    
    we use gradient descent in order to approximate U, S and V
    '''
    
    # init random approx for U, S, V 
    U = Array(random=True, dims=(m.dims[0],m.dims[0]), optimisable=True)
    
    S = Array(random=True, dims=m.dims, optimisable=True).diagonal()
    V = Array(random=True, dims=(m.dims[1],m.dims[1]), optimisable=True )
    
    
    # include in loss: loss for non-orthonormality, loss for non-zero off-diagonal elements and finally, for reconstruction approximation, 
    
    def loss_non_orthormality(X):
        sim = X.T() @ X
        # cols span orthonormal basis: sim ----> off-diagonal elements should be zero
        #                  ''              ----> diagonal elements should all be 1 
        
        sim_diag = sim.diagonal()
        idt = sim.identiy()
        non_normal = sim_diag - idt 
        sim_off_diag = sim - sim_diag
        
        l1 = sum([x**2 for x in list(flatten(sim_off_diag))])
        l2 = sum([x**2 for x in list(flatten(non_normal))])
        return l1 + l2  
    
    def loss_off_diagonal(S):
        S_diag = S.diagonal()
        non_zero = S - S_diag
        return sum([x**2 for x in list(flatten(non_zero))])
        
    
    def loss_reconstruction(m, U,S,V):
        return sum([x**2 for x in list(flatten(( m - (U @ S @ V.T()) ).values))])
        
    
    for _ in range(STEPS):
        
        loss = loss_reconstruction(m, U,S,V) + loss_off_diagonal(S) +  loss_non_orthormality(U) + loss_non_orthormality(V)
        loss.backwards()
        U = U - STEP_SIZE * U.grad()
        S = S - STEP_SIZE * S.grad()
        V = V - STEP_SIZE * V.grad()
        U._zero_grad()
        S._zero_grad()
        V._zero_grad()
        
        